In [ ]:
root_dir = "output"
method_name = "standard"
benchmark = "tabmwp"

model = "gpt-3.5-turbo"
eval_model = "gpt-4o-mini"
seed = 42
num_retries = 1
warming = [0.0]

In [ ]:
from agential.prompting.standard.prompting import Standard
method = Standard(
        llm=llm,
        benchmark=benchmark,
)

In [ ]:
from agential.core.llm import LLM
import os
llm = LLM(
        model, 
        api_key = open_api,
        temperature=0,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        seed=seed
)
eval_llm = LLM(
    eval_model,
    api_key = open_api,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    seed=seed
)

In [1]:
from datasets import load_dataset


data = load_dataset("Arietem/tabmwp")

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 1000/1000 [00:00<00:00, 2160.01 examples/s]


In [8]:
index = 0
question = data['train']['question'][index]
table_title = data['train']['table_title'][index]
table = data['train']['table'][index]
answer = data['train']["answer"][index]

answer = str(answer).replace(',', '')

question = '\n\n'.join(filter(None, [question, table_title, table]))


In [11]:
data['train'][-1]

{'question': "Joseph, an employee at Morrison's Convenience Store, looked at the sales of each of its soda products. How much more did the convenience store make from Nature's Nectar sales on Friday than on Thursday?",
 'choices': None,
 'answer': '2',
 'unit': '$',
 'table_title': 'Convenience store beverage sales',
 'table': "Beverage | Thursday | Friday\nNatural Goodness Juice | $18 | $10\nPearl's Soda | $7 | $4\nNature's Nectar | $2 | $4\nBrenner's Juices | $5 | $13",
 'table_for_pd': {'': None,
  'A Footlong shoe': None,
  'A Toes Knows shoe': None,
  'A little spicy': None,
  'A tent': None,
  'Activity': None,
  'Adult': None,
  'Amount spent': None,
  'Animal rights': None,
  'Apartment complex': None,
  'Apples': None,
  'April': None,
  'Arrive': None,
  'August': None,
  'Available Funds': None,
  'Bananas eaten': None,
  'Band member': None,
  'Baskets': None,
  'Batwing': None,
  'Beautician': None,
  'Beds': None,
  'Begin': None,
  'Beverage': ['Natural Goodness Juice',


In [ ]:
question = """Read the following table regarding "Bowling Scores" and then write Python code to answer a question:

Name | Score
Amanda | 117
Sam | 236
Irma | 144
Mike | 164

Question: Some friends went bowling and kept track of their scores. How many more points did Mike score than Irma?"""

In [9]:
print(question)

Some friends discussed the sizes of their coin collections. What is the mean of the numbers?

Coin collections

Name | Number of coins
Braden | 76
Camilla | 94
Rick | 86
Mary | 84
Hector | 80
Devin | 83
Emily | 82
Avery | 87


In [ ]:
from agential.eval.metrics.classification import f1, fuzzy_EM, llm_as_judge_eval, precision, recall, EM
import re

# Initialize score lists
em_scores = []

# Iterate through the first 10 instances in the training data
for index in range(min(len(data['train']['question']), 10)):
    # Extract data for the current instance
    question = data['train']['question'][index]
    table_title = data['train']['table_title'][index]
    table = data['train']['table'][index]
    answer = data['train']["answer"][index]

    # Clean up the answer
    answer = str(answer).replace(',', '')  # Remove commas, convert to string

    # Combine question components
    question = '\n'.join(filter(None, [question, table_title, table]))

    # Inference
    out = method.generate(
        question=question,
        key=answer,
        num_retries=num_retries,
        warming=warming
    )

    # Process the output
    code_str = out.answer.replace("```python", "").replace("```", "").strip()
    pred_answers, _ = safe_execute(code_string=code_str)
    pred_answer = str(pred_answers[0]).lower()

    # Determine the final predicted answer
    if any(word in pred_answer for word in ["yes", "true"]):
        pred_answer = "yes"
    elif any(word in pred_answer for word in ["no", "false"]):
        pred_answer = "no"

    print("Predicted answer:", pred_answer)
    print("Actual answer:", answer)

    # Evaluate correctness
    is_correct = int(EM(pred_answer, answer, is_numeric=True))
    em_scores.append(is_correct)
